### 可视化

In [ ]:
from utils.RFID_Dataset import RFID_Dataset

# 训练集
train_dataset=RFID_Dataset(
    r"data\RFID\dataset\temp",
    T=32,
    step=1,
    transform=None,
)
print(f"训练集的数据个数: {len(train_dataset)}")

In [ ]:
from utils.RFID_Dataset import RFID_Dataset

# 测试集
test_dataset=RFID_Dataset(
    r"./output/data",
    T=32,
    step=1,
    transform=None,
)
print(f"测试集的数据个数: {len(test_dataset)}")

In [ ]:
from utils.FeatureUtils import *

# 降维
pca = get_pca_model(train_dataset.datas, target=2)

train_output = use_pca_model(pca, train_dataset.datas)
test_output = use_pca_model(pca, test_dataset.datas)

In [ ]:
from utils.DataUtils.Visualization import plot_classes_scatter

plot_classes_scatter(train_output, train_dataset.labels,title="Train Data")
plot_classes_scatter(test_output, test_dataset.labels,title="Test Data")

In [ ]:
import numpy as np

# 数据分布对比
datas=np.concatenate([train_output,test_output],axis=0)
temp_labels=(test_dataset.labels)+3
labels=np.concatenate([train_dataset.labels,temp_labels],axis=0)
plot_classes_scatter(datas,labels)